# Step1 安装依赖

In [ ]:
!pip install gym
!pip install atari-py # 玩Gym的Atari游戏必装依赖，本次作业使用了Atari的Pong(乒乓球)环境
!pip install paddlepaddle==1.6.3
!pip install parl==1.3.1

     |████████████████████████████████| 90.9MB 107kB/s 
     |████████████████████████████████| 12.8MB 38.8MB/s 
     |████████████████████████████████| 24.8MB 1.3MB/s 
ERROR: umap-learn 0.4.6 has requirement scipy>=1.3.1, but you'll have scipy 1.2.1 which is incompatible.
ERROR: tensorflow 2.3.0 has requirement scipy==1.4.1, but you'll have scipy 1.2.1 which is incompatible.
ERROR: plotnine 0.6.0 has requirement matplotlib>=3.1.1, but you'll have matplotlib 2.2.4 which is incompatible.
ERROR: mizani 0.6.0 has requirement matplotlib>=3.1.1, but you'll have matplotlib 2.2.4 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
ERROR: paddlepaddle 1.6.3 has requirement numpy<=1.16.4,>=1.12, but you'll have numpy 1.18.5 which is incompatible.
  Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2
  Found existing installation: 

     |████████████████████████████████| 522kB 2.6MB/s 
     |████████████████████████████████| 471kB 12.0MB/s 
     |████████████████████████████████| 48.5MB 80kB/s 
     |████████████████████████████████| 4.3MB 44.3MB/s 
     |████████████████████████████████| 1.1MB 45.5MB/s 
     |████████████████████████████████| 5.3MB 38.1MB/s 
     |████████████████████████████████| 225kB 46.6MB/s 
     |████████████████████████████████| 174kB 50.0MB/s 
     |████████████████████████████████| 184kB 47.2MB/s 
     |████████████████████████████████| 81kB 7.2MB/s 
     |████████████████████████████████| 13.7MB 194kB/s 
     |████████████████████████████████| 4.9MB 42.6MB/s 
     |████████████████████████████████| 102kB 7.5MB/s 
     |████████████████████████████████| 276kB 44.6MB/s 
     |████████████████████████████████| 51kB 4.9MB/s 
     |████████████████████████████████| 71kB 6.5MB/s 
     |████████████████████████████████| 337kB 43.3MB/s 
  Created wheel for psutil: filename=psutil-5.7.3-cp36-cp

# Step2 导入依赖

In [ ]:
import os
import gym
import numpy as np

import paddle.fluid as fluid
import parl
from parl import layers
from parl.utils import logger

# Step3 设置超参数

In [ ]:
LEARNING_RATE = 1e-3

# Step4 搭建Model、Algorithm、Agent架构
* `Agent`把产生的数据传给`algorithm`，`algorithm`根据`model`的模型结构计算出`Loss`，使用`SGD`或者其他优化器不断的优化，`PARL`这种架构可以很方便的应用在各类深度强化学习问题中。

#### （1）Model
`Model`用来定义前向(`Forward`)网络，用户可以自由的定制自己的网络结构。

In [ ]:
class Model(parl.Model):
    def __init__(self, act_dim):
        self.act_dim = act_dim
        hid_size = act_dim * 10
        self.fc1 = layers.fc(size=hid_size, act='tanh')
        self.fc2 = layers.fc(size=hid_size, act='tanh')
        self.fc3 = layers.fc(size=self.act_dim, act='softmax')

    def forward(self, obs):  # 可直接用 model = Model(5); model(obs)调用
        out = self.fc1(obs)
        out = self.fc2(out)
        out = self.fc3(out)
        return out

#### （2）Algorithm
* `Algorithm` 定义了具体的算法来更新前向网络(`Model`)，也就是通过定义损失函数来更新`Model`，和算法相关的计算都放在`algorithm`中。

In [ ]:
from parl.algorithms import PolicyGradient # 直接从parl库中导入PolicyGradient算法，无需重复写算法

#### （3）Agent
* `Agent`负责算法与环境的交互，在交互过程中把生成的数据提供给`Algorithm`来更新模型(`Model`)，数据的预处理流程也一般定义在这里。

In [ ]:
class Agent(parl.Agent):
    def __init__(self, algorithm, obs_dim, act_dim):
        self.obs_dim = obs_dim
        self.act_dim = act_dim
        super(Agent, self).__init__(algorithm)

    def build_program(self):
        self.pred_program = fluid.Program()
        self.learn_program = fluid.Program()

        with fluid.program_guard(self.pred_program):  # 搭建计算图用于 预测动作，定义输入输出变量
            obs = layers.data(
                name='obs', shape=[self.obs_dim], dtype='float32')
            self.act_prob = self.alg.predict(obs)

        with fluid.program_guard(
                self.learn_program):  # 搭建计算图用于 更新policy网络，定义输入输出变量
            obs = layers.data(
                name='obs', shape=[self.obs_dim], dtype='float32')
            act = layers.data(name='act', shape=[1], dtype='int64')
            reward = layers.data(name='reward', shape=[], dtype='float32')
            self.cost = self.alg.learn(obs, act, reward)

    def sample(self, obs):
        obs = np.expand_dims(obs, axis=0)  # 增加一维维度
        act_prob = self.fluid_executor.run(
            self.pred_program,
            feed={'obs': obs.astype('float32')},
            fetch_list=[self.act_prob])[0]
        act_prob = np.squeeze(act_prob, axis=0)  # 减少一维维度
        act = np.random.choice(range(self.act_dim), p=act_prob)  # 根据动作概率选取动作
        return act

    def predict(self, obs):
        obs = np.expand_dims(obs, axis=0)
        act_prob = self.fluid_executor.run(
            self.pred_program,
            feed={'obs': obs.astype('float32')},
            fetch_list=[self.act_prob])[0]
        act_prob = np.squeeze(act_prob, axis=0)
        act = np.argmax(act_prob)  # 根据动作概率选择概率最高的动作
        return act

    def learn(self, obs, act, reward):
        act = np.expand_dims(act, axis=-1)
        feed = {
            'obs': obs.astype('float32'),
            'act': act.astype('int64'),
            'reward': reward.astype('float32')
        }
        cost = self.fluid_executor.run(
            self.learn_program, feed=feed, fetch_list=[self.cost])[0]
        return cost

### Step 5 Training && Test（训练&&测试）

In [ ]:
def run_episode(env, agent):
    obs_list, action_list, reward_list = [], [], []
    obs = env.reset()
    while True:
        obs = preprocess(obs) # from shape (210, 160, 3) to (100800,)
        obs_list.append(obs)
        action = agent.sample(obs) # 采样动作
        action_list.append(action)

        obs, reward, done, info = env.step(action)
        reward_list.append(reward)

        if done:
            break
    return obs_list, action_list, reward_list


# 评估 agent, 跑 5 个episode，求平均
def evaluate(env, agent, render=False):
    eval_reward = []
    for i in range(5):
        obs = env.reset()
        episode_reward = 0
        while True:
            obs = preprocess(obs) # from shape (210, 160, 3) to (100800,)
            action = agent.predict(obs) # 选取最优动作
            obs, reward, isOver, _ = env.step(action)
            episode_reward += reward
            if render:
                env.render()
            if isOver:
                break
        eval_reward.append(episode_reward)
    return np.mean(eval_reward)

### Step6 创建环境和Agent，启动训练，保存模型

In [ ]:
# Pong 图片预处理
def preprocess(image):
    """ 预处理 210x160x3 uint8 frame into 6400 (80x80) 1维 float vector """
    image = image[35:195] # 裁剪
    image = image[::2,::2,0] # 下采样，缩放2倍
    image[image == 144] = 0 # 擦除背景 (background type 1)
    image[image == 109] = 0 # 擦除背景 (background type 2)
    image[image != 0] = 1 # 转为灰度图，除了黑色外其他都是白色
    return image.astype(np.float).ravel()


# 根据一个episode的每个step的reward列表，计算每一个Step的Gt
def calc_reward_to_go(reward_list, gamma=0.99):
    """calculate discounted reward"""
    reward_arr = np.array(reward_list)
    for i in range(len(reward_arr) - 2, -1, -1):
        # G_t = r_t + γ·r_t+1 + ... = r_t + γ·G_t+1
        reward_arr[i] += gamma * reward_arr[i + 1]
    # normalize episode rewards
    reward_arr -= np.mean(reward_arr)
    reward_arr /= np.std(reward_arr)
    return reward_arr


# 创建环境
env = gym.make('Pong-v0')
obs_dim = 80 * 80
act_dim = env.action_space.n
logger.info('obs_dim {}, act_dim {}'.format(obs_dim, act_dim))

# 根据parl框架构建agent
######################################################################
######################################################################
#
# 4. 请参考课堂Demo构建 agent，嵌套Model, PolicyGradient, Agent
#
######################################################################
######################################################################
model = Model(act_dim)
alg = PolicyGradient(model, LEARNING_RATE)
agent = Agent(alg, obs_dim, act_dim)


# 加载模型
# if os.path.exists('./model.ckpt'):
#     agent.restore('./model.ckpt')

for i in range(1000):
    obs_list, action_list, reward_list = run_episode(env, agent)
    # if i % 10 == 0:
    #     logger.info("Train Episode {}, Reward Sum {}.".format(i, 
    #                                         sum(reward_list)))

    batch_obs = np.array(obs_list)
    batch_action = np.array(action_list)
    batch_reward = calc_reward_to_go(reward_list)

    agent.learn(batch_obs, batch_action, batch_reward)
    if (i + 1) % 100 == 0:
        total_reward = evaluate(env, agent, render=False)
        logger.info('Episode {}, Test reward: {}'.format(i + 1, 
                                            total_reward))

# save the parameters to ./model.ckpt
agent.save('./model.ckpt')

[11-03 08:11:27 MainThread @<ipython-input-13-85df6f6c0b74>:29] obs_dim 6400, act_dim 6
[11-03 08:11:27 MainThread @machine_info.py:88] Cannot find available GPU devices, using CPU now.
[11-03 08:11:27 MainThread @machine_info.py:88] Cannot find available GPU devices, using CPU now.
[11-03 08:18:28 MainThread @<ipython-input-13-85df6f6c0b74>:62] Episode 100, Test reward: -19.8
[11-03 08:28:06 MainThread @<ipython-input-13-85df6f6c0b74>:62] Episode 200, Test reward: -15.4
[11-03 08:40:55 MainThread @<ipython-input-13-85df6f6c0b74>:62] Episode 300, Test reward: -13.0
[11-03 08:55:22 MainThread @<ipython-input-13-85df6f6c0b74>:62] Episode 400, Test reward: -12.8
[11-03 09:10:34 MainThread @<ipython-input-13-85df6f6c0b74>:62] Episode 500, Test reward: -13.2
[11-03 09:25:20 MainThread @<ipython-input-13-85df6f6c0b74>:62] Episode 600, Test reward: -12.8
[11-03 09:40:18 MainThread @<ipython-input-13-85df6f6c0b74>:62] Episode 700, Test reward: -12.6
[11-03 09:55:21 MainThread @<ipython-input-1